In [ ]:
import re
import string
import pymorphy2
import pandas as pd

In [ ]:
aggregated = pd.read_csv('../data/marks_csv/aggregated.csv', index_col=None)

In [ ]:
regex_ru = re.compile('[^а-яА-Я]')

def get_ru_words(line: str):
    words = [word for word in regex_ru.sub(' ', line).lower().split() if word]
    return words
aggregated['ru_words'] = aggregated.input.apply(get_ru_words)
aggregated.head(2)

In [ ]:
from gensim.models.phrases import Phrases

DELIMITER = '_'
B_DELIMITER = DELIMITER.encode()

def train_phrases(sentences, n, min_count=30, threshold=50):
    if n < 2:
        raise ValueError("n < 2")
    
    grams = []
    transformed_corpus = []
    
    _words = sentences
    for ind in range(n - 1):
        gram = Phrases(_words, min_count=min_count, delimiter=B_DELIMITER, threshold=threshold)
        grams.append(gram)
        _words = gram[_words]
        transformed_corpus.append(_words)
            
    return grams, transformed_corpus

In [ ]:
grams, transformed_corpus = train_phrases(aggregated.ru_words, 5, threshold=25)

In [ ]:
phrases_by_groups = {name: list() for name in aggregated.output.unique()}
for index, (words, output) in enumerate(zip(transformed_corpus[-1], aggregated.output)):
    phrases = [word for word in words if DELIMITER in word]
    if phrases:
        phrases_by_groups[output].extend(phrases)

In [ ]:
from collections import Counter
counts = {name: Counter(values) for name, values in phrases_by_groups.items()}

In [ ]:
counts.keys()

In [ ]:
counts['GOOD'].most_common()

In [ ]:
counts['NEUTRAL'].most_common()

In [ ]:
counts['BAD'].most_common()